This notebook contains a template for basic startup operations for the Auxiliary telescope. At the begging of the night a user can run the startup procedure to prepare the telescope for observing. Omce it is done, remember to shutdown the notebook server. 

We start by loading all the libraries.

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [ ]:
from astropy.utils import iers
iers.conf.auto_download = False

Setup remotes for all the AT components

%%script bash 
make_idl_files.py ATMCS ATPtg ATAOS ATPneumatics ATHexapod ATDome ATDomeTrajectory

In [ ]:
d = salobj.Domain()

In [ ]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [ ]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

In [ ]:
ss = await atdome.evt_summaryState.next(flush=False, timeout=5)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await salobj.set_summary_state(atdome, salobj.State.ENABLED, settingsToApply="test.yaml")

In [ ]:
estop = await atdome.evt_emergencyStop.next(flush=False, timeout=60)

In [ ]:
estop.active, estop.private_sndStamp

In [ ]:
hb = await atdome.evt_heartbeat.next(flush=True, timeout=3)

In [ ]:
(estop.private_sndStamp-hb.private_sndStamp)/60.

Enable all components.

In [ ]:
await asyncio.gather(salobj.set_summary_state(atmcs, salobj.State.ENABLED, timeout=120),
                     salobj.set_summary_state(atptg, salobj.State.ENABLED),
                     salobj.set_summary_state(ataos, salobj.State.ENABLED, settingsToApply="measured_20190908.yaml"),
                     salobj.set_summary_state(atpne, salobj.State.ENABLED),
                     salobj.set_summary_state(athex, salobj.State.ENABLED, settingsToApply="Default1"),
                     salobj.set_summary_state(atdome, salobj.State.ENABLED, settingsToApply="test.yaml"),
                     salobj.set_summary_state(atdomtraj, salobj.State.ENABLED))

In [ ]:
await salobj.set_summary_state(atdomtraj, salobj.State.ENABLED)

In [ ]:
await asyncio.sleep(1.)
# await salobj.set_summary_state(atmcs, salobj.State.STANDBY, timeout=120)
await salobj.set_summary_state(atmcs, salobj.State.ENABLED, timeout=120)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(atdome, salobj.State.ENABLED, settingsToApply="test.yaml")

In [ ]:
await atdome.cmd_moveShutterMainDoor.set_start(open=True)

In [ ]:
await atpne.cmd_openM1Cover.start()

In [ ]:
await ataos.cmd_enableCorrection.set_start(m1=True)

In [ ]:
await atpne.cmd_openM1CellVents.start()

In [ ]:
err = await atdome.evt_errorCode.next(flush=False, timeout=5)

In [ ]:
err.errorReport

In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=90., elDegs=45)

In [ ]:
await atdome.cmd_homeAzimuth.start()

In [ ]:
await atdome.cmd_stopMotion.start()

In [ ]:
await atdome.cmd_moveAzimuth.set_start(azimuth=50)

In [ ]:
await asyncio.sleep(5)
inpos = atdome.evt_azimuthInPosition.get()

In [ ]:
inpos.inPosition

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(ataos, salobj.State.STANDBY, settingsToApply="measured_20190908.yaml")

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(ataos, salobj.State.ENABLED, settingsToApply="measured_20190908.yaml")

In [ ]:
await salobj.set_summary_state(athex, salobj.State.ENABLED, settingsToApply="Default1")

In [ ]:
await atmcs.tel_azEl_mountMotorEncoders.next(flush=True, timeout=5)